In [3]:
import polars as pl 
import pandas as pd 
import polars.selectors as cs


In [13]:
exp_vinho = (pl.read_csv("../data/ExpVinho.csv", separator=";") # lê os dados
             .select(~cs.starts_with("19") &                    # exclui colunas correspondentes ao século XX
                     ~cs.matches(r"(\b(200[0-6]))") &                   
                     ~cs.matches(r"(\b(200[0-6])_duplicated_0\b)")) # exclui colunas de 2000 a 2006    
             )
exp_vinho.head()

Id,País,2007,2007_duplicated_0,2008,2008_duplicated_0,2009,2009_duplicated_0,2010,2010_duplicated_0,2011,2011_duplicated_0,2012,2012_duplicated_0,2013,2013_duplicated_0,2014,2014_duplicated_0,2015,2015_duplicated_0,2016,2016_duplicated_0,2017,2017_duplicated_0,2018,2018_duplicated_0,2019,2019_duplicated_0,2020,2020_duplicated_0,2021,2021_duplicated_0,2022,2022_duplicated_0
i64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
1,"""Afeganistão""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11,46,0,0
2,"""África do Sul""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,26,95,4,21,0,0,0,0
3,"""Alemanha, Repú…",119512,238052,265742,429970,225086,393482,27715,138666,36070,144150,8189,56342,61699,265978,213348,761653,10680,44780,14012,68109,15467,87702,10794,45382,3660,25467,6261,32605,2698,6741,7630,45367
4,"""Angola""",25931,49753,25721,71083,54786,84235,33557,189891,13889,69001,2833,8861,1573,9300,12182,23124,1908,17089,7359,35390,10170,61680,477,709,345,1065,0,0,0,0,4068,4761
5,"""Anguilla""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [21]:
import re

# Define your regular expression pattern
pattern = r'((\d{4}|\d{4}_duplicated_0))'

# Your original list of strings
strings = ["2001", "2001_duplicated_0", "2002_name3", "other_text"]

# Define a function to use in re.sub()
def replace(match):
    year = match.group(1)
    name = match.group(2)
    new_name = f"{year}_qtd"
    return new_name

# Use re.sub() to apply the replacement
new_strings = [re.sub(pattern, replace, string) for string in strings]

# Print the renamed strings
for new_string in new_strings:
    print(new_string)


2001_qtd
2001_qtd_duplicated_0
2002_qtd_name3
other_text


In [23]:
old_cols = exp_vinho.columns
exp_vinho = (exp_vinho
    
    .rename({col: col.replace("_duplicated_0", "_val") for col in old_cols}) # renomeia as colunas para acrescentar o sufixo _val(US$)
    #.rename({col: re.sub(pattern, replace, col) for col in old_cols})
    
)

exp_vinho.head()

Id,País,2007,2007_val,2008,2008_val,2009,2009_val,2010,2010_val,2011,2011_val,2012,2012_val,2013,2013_val,2014,2014_val,2015,2015_val,2016,2016_val,2017,2017_val,2018,2018_val,2019,2019_val,2020,2020_val,2021,2021_val,2022,2022_val
i64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
1,"""Afeganistão""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11,46,0,0
2,"""África do Sul""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,26,95,4,21,0,0,0,0
3,"""Alemanha, Repú…",119512,238052,265742,429970,225086,393482,27715,138666,36070,144150,8189,56342,61699,265978,213348,761653,10680,44780,14012,68109,15467,87702,10794,45382,3660,25467,6261,32605,2698,6741,7630,45367
4,"""Angola""",25931,49753,25721,71083,54786,84235,33557,189891,13889,69001,2833,8861,1573,9300,12182,23124,1908,17089,7359,35390,10170,61680,477,709,345,1065,0,0,0,0,4068,4761
5,"""Anguilla""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
